In [1]:
import os 
import tqdm
import numpy as np
import pandas as pd
import nilearn as nil
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn.decomposition import CanICA
from nilearn.plotting import plot_prob_atlas

In [2]:
subj_ls = [
    "2357ZL",
    "2359ZL",
    "2360ZL",
    "2364ZL",
    "2365ZL",
    "2371ZL",
    "2377ZL",
    "2431ZL",
    "2486ZL",
    "2505ZL",
    "2523ZL",
    "2524ZL",
    "2528ZL",
    "2538ZL",
    "2540ZL",
    "2542ZL",
    "2544ZL",
    "2548ZL",
    "2553ZL",
    "2554ZL",
    "2555ZL",
    "2556ZL",
    "2561ZL",
    "2568ZL",
    "2570ZL",
    "2573ZL",
    "2577ZL",
    "2581ZL",
    "2582ZL",
    "2583ZL",
    "2584ZL",
    "2588ZL",
    "2590ZL",
    "2591ZL",
    "2592ZL",
    "2593ZL",
    "2594ZL",
    "2595ZL",
    "2596ZL",
    "2597ZL",
    "2599ZL",
    "2600ZL",
    "2602ZL",
    "2603ZL",
    "2604ZL",
    "2605ZL",
    "2607ZL",
    "2608ZL",
    "2609ZL",
    "2610ZL"
]

# ICA

In [5]:
for subj in tqdm.tqdm(subj_ls):
    nii=f"/mnt/d/doctoral_thesis/Rest/{subj}/func/s_scrub_nuis_nl_m_t_func.nii.gz"
    msk=f"/mnt/d/doctoral_thesis/Rest/{subj}/anat/bin_nl_brain_MPRage.nii.gz"

    ica_fig=f"/mnt/d/doctoral_thesis/Rest/{subj}/qc/ica/ICA_fig.jpg"
    ica_nii=f"/mnt/d/doctoral_thesis/Rest/{subj}/qc/ica/ICA_fig.nii.gz"
    
    try:
        os.mkdir(f"/mnt/d/doctoral_thesis/Rest/{subj}/qc")
        os.mkdir(f"/mnt/d/doctoral_thesis/Rest/{subj}/qc/ica")
        os.mkdir(f"/mnt/d/doctoral_thesis/Rest/{subj}/qc/motion")
    except FileExistsError: 
        print(f"Directory already exists: {subj}")

    ica=CanICA(mask=msk, smoothing_fwhm=None, n_components=4).fit(nii)
    plot_prob_atlas(ica.components_img_, output_file=ica_fig)
    ica.components_img_.to_filename(ica_nii)

100%|██████████| 50/50 [00:04<00:00, 10.09it/s]


# Motion Correlation

## Complete

In [ ]:
for subj in tqdm.tqdm(subj_ls):
    mask=f"/mnt/d/doctoral_thesis/Rest/{subj}/anat/bin_nl_brain_MPRage.nii.gz"

    nii_nuis        =f"/mnt/d/doctoral_thesis/Rest/{subj}/func/nuis_nl_m_t_func.nii.gz"
    nii_scrub_nuis  =f"/mnt/d/doctoral_thesis/Rest/{subj}/func/scrub_nuis_nl_m_t_func.nii.gz"
    nii_s_scrub_nuis=f"/mnt/d/doctoral_thesis/Rest/{subj}/func/s_scrub_nuis_nl_m_t_func.nii.gz"

    fd=f"/mnt/d/doctoral_thesis/Rest/{subj}/motion/fd.1D"
    fd_arr=np.genfromtxt(fd)

    # load mask
    mask_img=nib.load(mask).get_fdata()

    # mean time courses
    mean_nuis         = np.nanmean(nib.load(nii_nuis        ).get_fdata()[mask_img==1,:], axis=0)
    mean_scrub_nuis   = np.nanmean(nib.load(nii_scrub_nuis  ).get_fdata()[mask_img==1,:], axis=0)
    mean_s_scrub_nuis = np.nanmean(nib.load(nii_s_scrub_nuis).get_fdata()[mask_img==1,:], axis=0)

    # correlation
    corr_nuis         = np.corrcoef(fd_arr, mean_nuis        )[0,1]
    corr_scrub_nuis   = np.corrcoef(fd_arr, mean_scrub_nuis  )[0,1]
    corr_s_scrub_nuis = np.corrcoef(fd_arr, mean_s_scrub_nuis)[0,1]

    # prepare values
    corr_vals = [corr_nuis, corr_scrub_nuis, corr_s_scrub_nuis]
    corr_labs = ["nuis", "nuis +\nscrub", "nuis +\nscrub +\nsmooth"]

    # pandas dataframe
    corr_pd=pd.DataFrame(corr_vals, columns=[i.replace("\n", "") for i in corr_labs])

    # plotting
    plt.bar(corr_labs, corr_vals, color="grey", width=0.4)
    plt.xlabel("Preprocessing Steps")
    plt.ylabel("Correlation (Pearson's r)")
    plt.title("Correlation between global brain signal and framewise displacement (FD)")
    for i, v in enumerate(corr_vals):
        plt.text(i, v-(np.max(corr_vals)*0.2), str(np.round(v,2)), color="blue", fontweight="bold")

    # saving
    plt.savefig(f"/mnt/d/doctoral_thesis/Rest/{subj}/fd_correlation.jpg")
    corr_pd.to_csv(f"/mnt/d/doctoral_thesis/Rest/{subj}/fd_correlation.csv")

## Only Final

In [57]:
for subj in tqdm.tqdm(subj_ls):
    mask=f"/mnt/d/doctoral_thesis/Rest/{subj}/anat/bin_nl_brain_MPRage.nii.gz"

    nii_s_scrub_nuis=f"/mnt/d/doctoral_thesis/Rest/{subj}/func/s_scrub_nuis_nl_m_t_func.nii.gz"

    fd=f"/mnt/d/doctoral_thesis/Rest/{subj}/motion/fd.1D"
    fd_arr=np.genfromtxt(fd)

    # load mask
    mask_img=nib.load(mask).get_fdata()

    # mean time courses
    mean_s_scrub_nuis = np.nanmean(nib.load(nii_s_scrub_nuis).get_fdata()[mask_img==1,:], axis=0)

    # correlation
    corr_s_scrub_nuis = np.corrcoef(fd_arr, mean_s_scrub_nuis)[0,1]

    # prepare values
    corr_vals = [corr_s_scrub_nuis]
    corr_labs = ["nuis +\nscrub +\nsmooth"]

    # pandas dataframe
    corr_pd=pd.DataFrame(corr_vals, columns=[i.replace("\n", "") for i in corr_labs])

    # plotting
    plt.bar(corr_labs, corr_vals, color="grey", width=0.4)
    plt.ylabel("Pearson's r")
    plt.title("Correlation between GBS and FD")
    for i, v in enumerate(corr_vals):
        plt.text(i, v-(np.max(corr_vals)*0.2), str(np.round(v,2)), color="blue", fontweight="bold")

    # saving
    plt.savefig(f"/mnt/d/doctoral_thesis/Rest/{subj}/qc/motion/fd_correlation.jpg")
    corr_pd.to_csv(f"/mnt/d/doctoral_thesis/Rest/{subj}/qc/motion/fd_correlation.csv")
    
    plt.close()

100%|██████████| 50/50 [15:58<00:00, 19.17s/it]


<Figure size 432x288 with 0 Axes>

# Group Analysis

## Motion

In [109]:
grp_corrs=[float(pd.read_csv(f"/mnt/d/doctoral_thesis/Rest/{subj}/qc/motion/fd_correlation.csv", index_col=0).loc[0]) for subj in subj_ls ]
grp_labs="nuis + scrub + smooth"

In [110]:
["subjs", grp_labs]

['subjs', 'nuis + scrub + smooth']

In [116]:
grp_pd=pd.DataFrame(np.transpose([subj_ls, grp_corrs]), columns=["subjs", grp_labs])
grp_pd.to_csv("/mnt/d/doctoral_thesis/Rest/GROUP/motion/fd_correlation.csv", index=False)

In [128]:
# plotting
plt.bar(range(len(subj_ls)), grp_corrs, color="grey", width=.4)
plt.ylabel("Pearson's r")
plt.title("Correlation between GBS and FD")
plt.axhline( .3, color="red", linestyle="--")
plt.axhline(-.3, color="red", linestyle="--")
plt.savefig("/mnt/d/doctoral_thesis/Rest/GROUP/motion/fd_correlation.jpg")

plt.close()

## ICA

In [3]:
nii=[f"/mnt/d/doctoral_thesis/Rest/{subj}/func/s_scrub_nuis_nl_m_t_func.nii.gz" for subj in subj_ls]
msk="/mnt/d/doctoral_thesis/template/MNI152_T1_2mm_brain_mask.nii.gz"

ica_fig="/mnt/d/doctoral_thesis/Rest/GROUP/ica/ICA_fig.jpg"
ica_nii="/mnt/d/doctoral_thesis/Rest/GROUP/ica/ICA_fig.nii.gz"

ica=CanICA(mask=msk, smoothing_fwhm=None, n_components=4).fit(nii)
plot_prob_atlas(ica.components_img_, output_file=ica_fig)
ica.components_img_.to_filename(ica_nii)

/home/john/.local/lib/python3.8/site-packages/nilearn/plotting/img_plotting.py:300: FutureWarning: Default resolution of the MNI template will change from 2mm to 1mm in version 0.10.0
  anat_img = load_mni152_template()


### Outlier Detection

In [13]:
ica_scores=ica.score(nii, per_component=True)

In [15]:
ica_scores

array([0.01350413, 0.0231532 , 0.01251245, 0.01864819])